In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers.legacy import Adam
import matplotlib.pyplot as plt
import seaborn as sns
import optuna
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
df = pd.read_csv('aapl.csv')

In [3]:
df['date'] = pd.to_datetime(df['date'])
df.set_index('date',inplace=True)
df

,open,high,low,close,volume
date,,,,,
2016-01-04,23.353857,23.982029,23.215022,23.977476,270597600
2016-01-05,24.068514,24.091274,23.308337,23.376616,223164000
2016-01-06,22.887278,23.299232,22.730237,22.919142,273829600
2016-01-07,22.459392,22.789409,21.947296,21.951847,324377600
2016-01-08,22.429812,22.557267,22.022411,22.067930,283192000
...,...,...,...,...,...
2023-12-22,195.179993,195.410004,192.970001,193.600006,37122800
2023-12-26,193.610001,193.889999,192.830002,193.050003,28919300
2023-12-27,192.490005,193.500000,191.089996,193.149994,48087700


In [4]:
data = np.array(df)
data = np.reshape(df,(df.shape[0],-1))

scaler = MinMaxScaler()
data = scaler.fit_transform(data)

In [5]:
sequence_length = 50

def createTrainTestSplit(data, sequenceLength):
    X, Y = [], []
    for i in range(sequenceLength, len(data)):
        X.append(data[i-sequence_length:i, :])
        Y.append(data[i, 0])
    return np.array(X), np.array(Y)

feature, target = createTrainTestSplit(data,50)

xTrain,xTest,yTrain,yTest = train_test_split(feature,
                                                target,
                                                test_size=0.2,
                                                random_state=42,
                                                shuffle=False)
print(xTrain.shape,yTrain.size)
print(xTest.shape, yTest.shape)

(1569, 50, 5) 1569
(393, 50, 5) (393,)


In [8]:
def objective(trial):


    nLstmLayers = trial.suggest_int('n_lstm_layers', 1, 6)
    model = Sequential()
    for i in range(nLstmLayers):
        numHidden = trial.suggest_int(f'n_lstm_units_l{i}', 4, 512)
        if i == 0:
            model.add(LSTM(numHidden, return_sequences=True, input_shape=(xTrain.shape[1], xTrain.shape[2])))
        else:
            model.add(LSTM(numHidden,return_sequences=True))
    numLastLSTM = trial.suggest_int(f'n__lstm_units_last{i+1}',4,64)
    model.add(LSTM(numLastLSTM))

    nDenseLayers = trial.suggest_int(f'n_dense_layers',1,4)
    for i in range(nDenseLayers):
        numHidden = trial.suggest_int(f'n_units_d{i}', 4, 512)
        model.add(Dense(numHidden))

    model.add(Dense(1))
    learningRate = trial.suggest_float(
        "learning_rate_init", 1e-5, 1e-3, log=True
    )
    model.compile(optimizer=Adam(learning_rate=learningRate), loss='mean_squared_error')

    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min')
    batchSize = trial.suggest_categorical('batch_size', [4,16,64])
    history = model.fit(xTrain, yTrain, batch_size=batchSize, verbose=1,epochs=100, validation_data=(xTest, yTest), callbacks=[earlyStopping, reduceLr])

    valLoss = history.history['val_loss'][-1]

    return valLoss

In [9]:
storageUrl = "sqlite:///multivariateStockStudy.db"
study = optuna.create_study(study_name="multivariateStockStudy", storage=storageUrl, direction="minimize", load_if_exists=True)
study.optimize(objective, n_trials=5)

[I 2024-01-20 19:51:15,282] A new study created in RDB with name: multivariateStockStudy


Epoch 1/100
25/25 [==============================] - 11s 107ms/step - loss: 0.0195 - val_loss: 0.0062 - lr: 2.5361e-04
Epoch 2/100
25/25 [==============================] - 1s 31ms/step - loss: 0.0020 - val_loss: 0.0039 - lr: 2.5361e-04
Epoch 3/100
25/25 [==============================] - 1s 31ms/step - loss: 0.0011 - val_loss: 0.0043 - lr: 2.5361e-04
Epoch 4/100
25/25 [==============================] - 1s 31ms/step - loss: 9.9762e-04 - val_loss: 0.0033 - lr: 2.5361e-04
Epoch 5/100
25/25 [==============================] - 1s 31ms/step - loss: 9.8626e-04 - val_loss: 0.0031 - lr: 2.5361e-04
Epoch 6/100
25/25 [==============================] - 1s 30ms/step - loss: 9.3438e-04 - val_loss: 0.0046 - lr: 2.5361e-04
Epoch 7/100
25/25 [==============================] - 1s 31ms/step - loss: 9.0542e-04 - val_loss: 0.0027 - lr: 2.5361e-04
Epoch 8/100
25/25 [==============================] - 1s 34ms/step - loss: 8.4712e-04 - val_loss: 0.0026 - lr: 2.5361e-04
Epoch 9/100
25/25 [=======================

[I 2024-01-20 19:52:06,113] Trial 0 finished with value: 0.00154775264672935 and parameters: {'n_lstm_layers': 4, 'n_lstm_units_l0': 289, 'n_lstm_units_l1': 276, 'n_lstm_units_l2': 465, 'n_lstm_units_l3': 29, 'n__lstm_units_last4': 25, 'n_dense_layers': 3, 'n_units_d0': 476, 'n_units_d1': 383, 'n_units_d2': 482, 'learning_rate_init': 0.00025361255719943477, 'batch_size': 64}. Best is trial 0 with value: 0.00154775264672935.


Epoch 1/100
99/99 [==============================] - 12s 43ms/step - loss: 0.0621 - val_loss: 0.0123 - lr: 1.1083e-05
Epoch 2/100
99/99 [==============================] - 2s 19ms/step - loss: 0.0016 - val_loss: 0.0058 - lr: 1.1083e-05
Epoch 3/100
99/99 [==============================] - 2s 19ms/step - loss: 0.0014 - val_loss: 0.0049 - lr: 1.1083e-05
Epoch 4/100
99/99 [==============================] - 2s 19ms/step - loss: 0.0014 - val_loss: 0.0053 - lr: 1.1083e-05
Epoch 5/100
99/99 [==============================] - 2s 22ms/step - loss: 0.0013 - val_loss: 0.0047 - lr: 1.1083e-05
Epoch 6/100
99/99 [==============================] - 2s 24ms/step - loss: 0.0013 - val_loss: 0.0054 - lr: 1.1083e-05
Epoch 7/100
99/99 [==============================] - 2s 19ms/step - loss: 0.0013 - val_loss: 0.0061 - lr: 1.1083e-05
Epoch 8/100
99/99 [==============================] - 2s 19ms/step - loss: 0.0013 - val_loss: 0.0049 - lr: 1.1083e-05
Epoch 9/100
99/99 [==============================] - 2s 19ms/st

[I 2024-01-20 19:53:25,420] Trial 1 finished with value: 0.00377241475507617 and parameters: {'n_lstm_layers': 5, 'n_lstm_units_l0': 337, 'n_lstm_units_l1': 252, 'n_lstm_units_l2': 130, 'n_lstm_units_l3': 217, 'n_lstm_units_l4': 128, 'n__lstm_units_last5': 9, 'n_dense_layers': 4, 'n_units_d0': 72, 'n_units_d1': 428, 'n_units_d2': 214, 'n_units_d3': 339, 'learning_rate_init': 1.1083318649760951e-05, 'batch_size': 16}. Best is trial 0 with value: 0.00154775264672935.


...

[I 2024-01-20 19:56:06,591] Trial 2 finished with value: 0.0005817337660118937 and parameters: {'n_lstm_layers': 6, 'n_lstm_units_l0': 182, 'n_lstm_units_l1': 136, 'n_lstm_units_l2': 273, 'n_lstm_units_l3': 351, 'n_lstm_units_l4': 47, 'n_lstm_units_l5': 246, 'n__lstm_units_last6': 55, 'n_dense_layers': 1, 'n_units_d0': 139, 'learning_rate_init': 0.00010688241413055936, 'batch_size': 16}. Best is trial 2 with value: 0.0005817337660118937.


Epoch 1/100
393/393 [==============================] - 9s 15ms/step - loss: 0.0021 - val_loss: 0.0017 - lr: 4.1982e-04
Epoch 2/100
393/393 [==============================] - 4s 10ms/step - loss: 5.1696e-04 - val_loss: 6.8411e-04 - lr: 4.1982e-04
Epoch 3/100
393/393 [==============================] - 4s 10ms/step - loss: 4.5506e-04 - val_loss: 8.5799e-04 - lr: 4.1982e-04
Epoch 4/100
393/393 [==============================] - 3s 8ms/step - loss: 3.9163e-04 - val_loss: 5.1986e-04 - lr: 4.1982e-04
Epoch 5/100
393/393 [==============================] - 3s 8ms/step - loss: 3.6103e-04 - val_loss: 4.1968e-04 - lr: 4.1982e-04
Epoch 6/100
393/393 [==============================] - 3s 8ms/step - loss: 2.9606e-04 - val_loss: 3.8631e-04 - lr: 4.1982e-04
Epoch 7/100
393/393 [==============================] - 4s 10ms/step - loss: 2.5118e-04 - val_loss: 3.1516e-04 - lr: 4.1982e-04
Epoch 8/100
393/393 [==============================] - 3s 8ms/step - loss: 2.4583e-04 - val_loss: 3.5775e-04 - lr: 4.1982e

[I 2024-01-20 19:58:22,637] Trial 3 finished with value: 0.00011599590652622283 and parameters: {'n_lstm_layers': 1, 'n_lstm_units_l0': 262, 'n__lstm_units_last1': 63, 'n_dense_layers': 1, 'n_units_d0': 181, 'learning_rate_init': 0.0004198174937214897, 'batch_size': 4}. Best is trial 3 with value: 0.00011599590652622283.


Epoch 1/100
25/25 [==============================] - 12s 138ms/step - loss: 0.0417 - val_loss: 0.0173 - lr: 7.2117e-05
Epoch 2/100
25/25 [==============================] - 1s 49ms/step - loss: 0.0023 - val_loss: 0.0049 - lr: 7.2117e-05
Epoch 3/100
25/25 [==============================] - 1s 48ms/step - loss: 0.0014 - val_loss: 0.0054 - lr: 7.2117e-05
Epoch 4/100
25/25 [==============================] - 1s 48ms/step - loss: 0.0013 - val_loss: 0.0044 - lr: 7.2117e-05
Epoch 5/100
25/25 [==============================] - 1s 49ms/step - loss: 0.0014 - val_loss: 0.0042 - lr: 7.2117e-05
Epoch 6/100
25/25 [==============================] - 1s 48ms/step - loss: 0.0013 - val_loss: 0.0042 - lr: 7.2117e-05
Epoch 7/100
25/25 [==============================] - 1s 49ms/step - loss: 0.0013 - val_loss: 0.0039 - lr: 7.2117e-05
Epoch 8/100
25/25 [==============================] - 1s 50ms/step - loss: 0.0012 - val_loss: 0.0059 - lr: 7.2117e-05
Epoch 9/100
25/25 [==============================] - 1s 51ms/s

[I 2024-01-20 19:59:14,824] Trial 4 finished with value: 0.0038687221240252256 and parameters: {'n_lstm_layers': 5, 'n_lstm_units_l0': 320, 'n_lstm_units_l1': 133, 'n_lstm_units_l2': 485, 'n_lstm_units_l3': 342, 'n_lstm_units_l4': 362, 'n__lstm_units_last5': 19, 'n_dense_layers': 3, 'n_units_d0': 109, 'n_units_d1': 404, 'n_units_d2': 278, 'learning_rate_init': 7.211749757727082e-05, 'batch_size': 64}. Best is trial 3 with value: 0.00011599590652622283.


In [7]:
study = optuna.load_study(study_name="multivariateStockStudy", storage="sqlite:///Multivariate.db")
best_trial = study.best_trial

print("Best trial ID: {}".format(best_trial.number))
print("Value of the best objective (lowest validation loss): {}".format(best_trial.value))


print("Best hyperparameters:")
for key, value in best_trial.params.items():
    print("{}: {}".format(key, value))


Best trial ID: 3
Value of the best objective (lowest validation loss): 0.00011599590652622283
Best hyperparameters:
n_lstm_layers: 1
n_lstm_units_l0: 262
n__lstm_units_last1: 63
n_dense_layers: 1
n_units_d0: 181
learning_rate_init: 0.0004198174937214897
batch_size: 4


In [8]:
def buildModel(trial):

    nLstmLayers = trial.suggest_int('n_lstm_layers', 1, 6)
    model = Sequential()
    for i in range(nLstmLayers):
        numHidden = trial.suggest_int(f'n_lstm_units_l{i}', 4, 512)
        if i == 0:
            model.add(LSTM(numHidden, return_sequences=True, input_shape=(xTrain.shape[1], xTrain.shape[2])))
        else:
            model.add(LSTM(numHidden,return_sequences=True))
    numLastLSTM = trial.suggest_int(f'n__lstm_units_last{i+1}',4,64)
    model.add(LSTM(numLastLSTM))

    nDenseLayers = trial.suggest_int(f'n_dense_layers',1,4)
    for i in range(nDenseLayers):
        numHidden = trial.suggest_int(f'n_units_d{i}', 4, 512)
        model.add(Dense(numHidden))

    model.add(Dense(1))
    learningRate = trial.suggest_float(
        "learning_rate_init", 1e-5, 1e-3, log=True
    )
    model.compile(optimizer=Adam(learning_rate=learningRate), loss='mean_squared_error')
    earlyStopping = EarlyStopping(monitor='val_loss', patience=10, verbose=1, mode='min', restore_best_weights=True)
    reduceLr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, verbose=1, mode='min')
    batchSize = trial.suggest_categorical('batch_size', [4,16,64])
    history = model.fit(xTrain, yTrain, batch_size=batchSize, verbose=1,epochs=100, validation_data=(xTest, yTest), callbacks=[earlyStopping, reduceLr])

    return model,history

In [9]:
model,history = buildModel(best_trial)

Epoch 1/100
393/393 [==============================] - 14s 32ms/step - loss: 0.0016 - val_loss: 0.0011 - lr: 4.1982e-04
Epoch 2/100
393/393 [==============================] - 12s 30ms/step - loss: 5.6461e-04 - val_loss: 7.9066e-04 - lr: 4.1982e-04
Epoch 3/100
393/393 [==============================] - 11s 28ms/step - loss: 5.5402e-04 - val_loss: 9.6335e-04 - lr: 4.1982e-04
Epoch 4/100
393/393 [==============================] - 11s 28ms/step - loss: 4.3506e-04 - val_loss: 5.0734e-04 - lr: 4.1982e-04
Epoch 5/100
393/393 [==============================] - 11s 27ms/step - loss: 3.6413e-04 - val_loss: 9.4981e-04 - lr: 4.1982e-04
Epoch 6/100
393/393 [==============================] - 12s 30ms/step - loss: 2.6574e-04 - val_loss: 0.0010 - lr: 4.1982e-04
Epoch 7/100
393/393 [==============================] - 11s 28ms/step - loss: 2.9329e-04 - val_loss: 0.0038 - lr: 4.1982e-04
Epoch 8/100
393/393 [==============================] - 11s 28ms/step - loss: 2.3973e-04 - val_loss: 4.5424e-04 - lr: 4.1

In [11]:
predictions = model.predict(xTest)

df.reset_index(inplace=True)
dates = df['date'].values[-len(yTest):]


data = {
    "date": dates,
    "predictedOpen": predictions.flatten(),
    "open": yTest
}

dfPredicted = pd.DataFrame(data)
dfPredicted
dfPredicted["date"] = pd.to_datetime(dfPredicted["date"])
# dfPredicted['predictedOpen'] = scaler.inverse_transform(dfPredicted['predictedOpen'].reshape(-1, 1))
# dfPredicted['open'] = scaler.inverse_transform(dfPredicted['open'].reshape(-1, 1))


MAE = mean_absolute_error(dfPredicted["predictedOpen"], dfPredicted["open"])
print(f'Mean Absolute Error (MAE): {np.round(MAE, 2)}')

MSE = mean_squared_error( dfPredicted["open"],dfPredicted["predictedOpen"])
print(f'Mean Squared Error (MAE): {np.round(MSE, 2)}')

MAPE = np.mean(np.abs((dfPredicted["open"] - dfPredicted["predictedOpen"]) / dfPredicted["open"])) * 100
print(f'Mean Absolute Percentage Error (MAPE): {np.round(MAPE, 2)} %')


MDAPE = np.median(np.abs((dfPredicted["open"] - dfPredicted["predictedOpen"]) / dfPredicted["open"])) * 100
print(f'Median Absolute Percentage Error (MDAPE): {np.round(MDAPE, 2)} %')


13/13 [==============================] - 0s 23ms/step
Mean Absolute Error (MAE): 0.01
Mean Squared Error (MAE): 0.0
Mean Absolute Percentage Error (MAPE): 1.06 %
Median Absolute Percentage Error (MDAPE): 0.8 %


In [12]:
historyDf = pd.DataFrame(history.history).reset_index()
historyDf.rename(columns={'index':'epoch'},inplace=True)
fig = go.Figure()
fig.add_trace(go.Scatter(x=historyDf["epoch"], y=historyDf["loss"],name='Train Loss',line=dict(color='blue', width=1)))
fig.add_trace(go.Scatter(x=historyDf["epoch"], y=historyDf["val_loss"],name='Validation Loss',line=dict(color='red', width=1)))
fig.update_layout(height=400, width=1200, title_text=f"Learning Curve", showlegend=True)

In [13]:
fig = make_subplots()


fig.add_trace(go.Scatter(x=dfPredicted['date'], y=dfPredicted['open'],
                         mode='lines', name='Actual'))


fig.add_trace(go.Scatter(x=dfPredicted['date'], y=dfPredicted['predictedOpen'],
                         mode='lines', name='Predicted'))


fig.add_trace(go.Scatter(x=dfPredicted['date'].tolist() + dfPredicted['date'].tolist()[::-1],
                         y=dfPredicted['open'].tolist() + dfPredicted['predictedOpen'].tolist()[::-1],
                         fill='toself',
                         fillcolor='grey',
                         opacity=0.3,
                         line=dict(color='rgba(255,255,255,0)'),
                         showlegend=False))


fig.update_layout(title='Predicted vs Actual Over Test Data',
                  xaxis_title='Date',
                  yaxis_title='Value',
                  xaxis=dict(showgrid=False),
                  yaxis=dict(showgrid=False),
                  plot_bgcolor="white")


fig.update_xaxes(tickangle=45)


fig.show()